In [3]:
import numpy as np
import pandas as pd

In [94]:
pred = pd.DataFrame()
pred['srch_id'] = [1,1,1,1,1,2,2,2,3,3,3,3,3,3,3]
pred['prop_id'] = [1,2,3,5,4,1,2,3,7,6,5,4,3,2,1]

truth = pd.DataFrame()
truth['srch_id'] = [1,1,1,1,1,2,2,2,3,3,3,3,3,3,3]
truth['prop_id'] = [1,2,3,4,5,1,2,3,1,2,3,4,5,6,7]
truth['booking_bool'] = [1,0,0,0,0,1,0,0,1,0,0,0,0,0,0]
truth['click_bool'] = [1,1,1,1,0,1,1,0,1,1,1,0,0,0,0]

In [96]:
def get_score(row, booking_dict, click_dict):
    prop_id = row['prop_id']
    if booking_dict[prop_id]:
        return 5
    elif click_dict[prop_id]:
        return 1
    else:
        return 0
    
def calculate_DCG(row, booking_dict, click_dict):
    return (2**get_score(row, booking_dict, click_dict) - 1)/(np.log2(row.name + 2))

def ideal_ranking(ground_truth):
    ideal_ranking = ground_truth.sort_values(by=['booking_bool', 'click_bool'], ascending=False)
    return ideal_ranking.reset_index(drop=True)
    
def nDCG(prediction, ground_truth):
    prediction.reset_index(drop=True, inplace=True)
    booking_dict = dict(zip(ground_truth.prop_id.values, ground_truth.booking_bool.values))
    click_dict = dict(zip(ground_truth.prop_id.values, ground_truth.click_bool.values))
    dcg = sum(prediction.apply(calculate_DCG, args=(booking_dict, click_dict), axis=1))
    idcg = sum(ideal_ranking(ground_truth).apply(calculate_DCG, args=(booking_dict, click_dict), axis=1))
    return (dcg, idcg, dcg/idcg)

In [97]:
for srch_id in range(1,4):
    print(nDCG(pred[pred.srch_id==srch_id], truth[truth.srch_id==srch_id]))

(32.517782560806, 32.56160631164485, 0.9986541281035272)
(31.630929753571458, 31.630929753571458, 1.0)
(11.076393327675898, 32.13092975357146, 0.3447268228036483)
